## filtrado

In [2]:
from obspy import read, read_inventory,UTCDateTime
from obspy.geodetics.base import calc_vincenty_inverse, locations2degrees
from obspy.taup.tau import TauPyModel
import os

In [3]:
## cargamos las carpetas asi, pero luego en la interfaz sera automatico
ruta = os.getcwd()
ruta_w = ruta + "/"+"datos/FIJI ISLANDS REGION2018-8-19/waveforms"
ruta_s = ruta + "/"+"datos/FIJI ISLANDS REGION2018-8-19/stations"

In [5]:
#leemos los archivos waveforms
st = read(ruta_w+"/*.mseed")

In [6]:
#leemos los xml
XML = os.listdir(ruta_s)
XML = sorted(XML)

In [7]:
c = 0
# un filtro para frecuencias muy altas y bajas
pre_filt = [0.001, 0.005, 10, 20]
dist = []
az = []
baz = []
canal = []
great_circle = []
arrivals = []
taup = TauPyModel()

In [8]:
st.traces[0].__dict__

{'stats':          network: AU
         station: KNRA
        location: 00
         channel: BHZ
       starttime: 2018-08-19T00:18:40.675000Z
         endtime: 2018-08-19T01:19:40.650000Z
   sampling_rate: 40.0
           delta: 0.025
            npts: 146400
           calib: 1.0
         _format: MSEED
           mseed: AttribDict({'dataquality': 'M', 'number_of_records': 600, 'encoding': 'STEIM1', 'byteorder': '>', 'record_length': 512, 'filesize': 307200}),
 'data': array([ 2219,  2245,  2259, ..., 14253, 14232, 14208])}

In [30]:
ruta_w = ruta_w.replace("waveforms","info.txt")
archivo = open(ruta_w)
array =[]
for element in archivo:
    array.append(element.replace("\n",""))
#lat
#lon
#time

In [31]:
lat_e = array[0]
lon_e = array[1]
time = UTCDateTime(array[2])

## Remove Respuesta Instrumento

In [29]:
for resp in XML:
    print(resp)  # para cachar donde pueda cagar el script
    # leemos la info del instrumento
    inv = read_inventory(str(resp))
    datachannel = inv.get_contents()
    channel = datachannel['channels']  # extraer el canal correspondiente
    # sacamos las coordenadas del canal  para t=tfiji
    datacoor = inv.get_coordinates(channel[0], origin_time)
    coords = [datacoor['latitude'], datacoor[
        'longitude'], datacoor['local_depth']]
    # CALCULAR DISTANCIA Y AZIMUTHS EVENTO-ESTACION
    [disti, azi, bazi] = calc_vincenty_inverse(
        epic[0], epic[1], coords[0], coords[1])
    greatcirc = locations2degrees(epic[0], epic[1], coords[0], coords[1])
    # Arriv=taup.get_travel_times(source_depth_in_km=source_depth,distance_in_degree=greatcirc,phase_list=['P'],\
    #                      receiver_depth_in_km=coords[2])
    # arrivals.append(Arriv)

    # lol=TauPyModel.get_travel_times(source_depth_in_km=141,distance_in_degree=44,phase_list='P',receiver_depth_in_km=0.0)

    # guardar todo en listas (quizá  en pandas se vería mejor?)
    dist.append(disti)
    az.append(azi)
    baz.append(bazi)
    canal.append(channel[0])
    # remover respuesta del instrumento
    st[c].remove_response(inventory=inv, pre_filt=pre_filt,
                          output="VEL", plot=False)
    # aplicar un lowpass
    st[c].filter('lowpass', freq=0.2, corners=2, zerophase=True)
    c += 1


AU.KNRA.xml


OSError: Error reading file 'AU.KNRA.xml': failed to load external entity "AU.KNRA.xml"

## Graficar 